In [ ]:
## TOOL UPDATED: Feb 23 2023, Henrik Loecke

##Double click this cell to see full description.

##You must restart the kernel after updating System_Assessment_Variables!

<!-- 

Workflow breakdown

1: Generate model. This also creates csv file Deficits.csv
2: Create Theory_ADWF.xlsx. This is a manual process which must be improved.



To run this notebook, click menu Cell -> Run All

User input has been moved away from this notebook so it can easily be replaced by new versions.
 
Please open System_Assessment_Variables, in the same folder as this notebook, to edit user input there.

All variables with path must start with 'r', e.g. r'C:\Projects'

It must contain the following variables:

model_area:                          Short area name like 'VSA' or LISA'
result_specs_csv:                    CSV file linking network and runoff result file. Only needed if runoff imported.
ps_specs_csv:                        CSV file containing PS firm and station capacities.
summation_csv:                       Summarizes which elements are to be summed up, mostly used for outfalls.
summation_ps_csv:                    Summarizes which elements are to be summed up for PS assessment.
outfall_csv:                         Summarizes all outfalls that are to have overflows displayed on the map.    
result_folder:                       Folder path of result files.
map_folder:                          Folder path where maps are generated and where this tool returns output.

 -->


In [36]:
import mikeio
import mikeio1d
from mikeio1d.res1d import Res1D
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objects as go
from ipywidgets import widgets

In [37]:
#FSA Port Coquitlam
wwtp_pipe = '52458'
specs = []

specs.append([3,11.326,'FSA_DWF_2060pop_SealPoco_FN_VFD_3ADWF_Base.res1d','Annacis','VFD'])

df_theory_sheet = r"J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\Port_Coquitlam_PS_VFD\Model_3ADWF\SA_Maps\Theory_ADWF.xlsx"
result_folder = r"J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\Port_Coquitlam_PS_VFD\Model_3ADWF"
map_folder = r"J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\Port_Coquitlam_PS_VFD\Model_3ADWF\SA_Maps"
dwf_file_path = r"J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\Port_Coquitlam_PS_VFD\Model_3ADWF\FSA_DWF_2060pop_SealPoco_FN_VFD_Base.res1d"
runoff_results = [] #Must be in order, smallest return period at top
runoff_results.append([2,'FSA_Runoff_EX-2yr-24hr-AES_BaseRR.res1d'])
runoff_results.append([5,'FSA_Runoff_EX-5yr-24hr-AES_BaseRR.res1d'])
runoff_results.append([10,'FSA_Runoff_EX-10yr-24hr-AES_BaseRR.res1d'])
runoff_results.append([25,'FSA_Runoff_EX-25yr-24hr-AES_BaseRR.res1d'])
runoff_results.append([50,'FSA_Runoff_EX-50yr-24hr-AES_BaseRR.res1d'])
runoff_results.append([100,'FSA_Runoff_EX-100yr-24hr-AES_BaseRR.res1d'])
runoff_results.append([200,'FSA_Runoff_EX-200yr-24hr-AES_BaseRR.res1d'])


# #FSA
# wwtp_pipe = '52458'
# specs = []
# specs.append([2.5,5.886,'FSA_2p5ADWF_2020pop_Base.res1d','Annacis',''])
# specs.append([3,8.607,'FSA_3ADWF_2020pop_Base.res1d','Annacis',''])
# specs.append([4,14.050,'FSA_4ADWF_2020pop_Base.res1d',"Annacis",''])

# specs.append([2.5,5.886,'FSA_2p5ADWF_2020pop_VFD_Base.res1d','Annacis','VFD'])
# specs.append([3,8.607,'FSA_3ADWF_2020pop_VFD_Base.res1d','Annacis','VFD'])
# specs.append([4,14.050,'FSA_4ADWF_2020pop_VFD_Base.res1d',"Annacis",'VFD'])
# specs.append([5,19.492,'FSA_5ADWF_2020pop_VFD_Base.res1d',"Annacis",'VFD'])

# df_theory_sheet = r"J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\Threes_Times_ADWF\Model\SA_Maps\Theory_ADWF.xlsx"
# result_folder = r"J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\Threes_Times_ADWF\Model"
# map_folder = r"J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\Threes_Times_ADWF\Model\SA_Maps"
# dwf_file_path = r"J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\Allways_Latest_Master_Model_Simulations\Model\FSA_DWF_2021-07-23_4d_2020pop_Base.res1d"
# runoff_results = [] #Must be in order, smallest return period at top
# runoff_results.append([2,'FSA_Runoff_EX-2yr-24hr-AES_BaseRR.res1d'])
# runoff_results.append([5,'FSA_Runoff_EX-5yr-24hr-AES_BaseRR.res1d'])
# runoff_results.append([10,'FSA_Runoff_EX-10yr-24hr-AES_BaseRR.res1d'])
# runoff_results.append([25,'FSA_Runoff_EX-25yr-24hr-AES_BaseRR.res1d'])
# runoff_results.append([50,'FSA_Runoff_EX-50yr-24hr-AES_BaseRR.res1d'])
# runoff_results.append([100,'FSA_Runoff_EX-100yr-24hr-AES_BaseRR.res1d'])
# runoff_results.append([200,'FSA_Runoff_EX-200yr-24hr-AES_BaseRR.res1d'])


# #NSSA
# wwtp_pipe = '54959_a'
# specs = []
# specs.append([3,1.107,'NSSA_3ADWF_2018pop_Base.res1d',"Lion's Gate",''])
# specs.append([3.5,1.466,'NSSA_3p5ADWF_2018pop_Base.res1d',"Lion's Gate",''])
# specs.append([4,1.825,'NSSA_4ADWF_2018pop_Base.res1d',"Lion's Gate",''])
# specs.append([5,2.542,'NSSA_5ADWF_2018pop_Base.res1d',"Lion's Gate",''])

# specs.append([3,1.107,'NSSA_3ADWF_2018pop_VFD_Base.res1d',"Lion's Gate",'VFD'])
# specs.append([3.5,1.466,'NSSA_3p5ADWF_2018pop_VFD_Base.res1d',"Lion's Gate",'VFD'])
# specs.append([4,1.825,'NSSA_4ADWF_2018pop_VFD_Base.res1d',"Lion's Gate",'VFD'])
# specs.append([5,2.542,'NSSA_5ADWF_2018pop_VFD_Base.res1d',"Lion's Gate",'VFD'])

# df_theory_sheet = r"J:\SEWER_AREA_MODELS\NSSA\03_SIMULATION_WORK\X-ADWF\Model\SA_Maps\Theory_ADWF.xlsx"
# result_folder = r"J:\SEWER_AREA_MODELS\NSSA\03_SIMULATION_WORK\X-ADWF\Model"
# map_folder = r"J:\SEWER_AREA_MODELS\NSSA\03_SIMULATION_WORK\X-ADWF\Model\SA_Maps"
# dwf_file_path = r"J:\SEWER_AREA_MODELS\NSSA\03_SIMULATION_WORK\X-ADWF\Model\NSSA_DWF_2018-07-26_4d_2018pop_Base.res1d"
# runoff_results = [] #Must be in order, smallest return period at top
# runoff_results.append([2,'NSSA_Runoff_EX-2yr-24hr-AES_BaseRR.res1d'])
# runoff_results.append([5,'NSSA_Runoff_EX-5yr-24hr-AES_BaseRR.res1d'])
# runoff_results.append([10,'NSSA_Runoff_EX-10yr-24hr-AES_BaseRR.res1d'])
# runoff_results.append([25,'NSSA_Runoff_EX-25yr-24hr-AES_BaseRR.res1d'])
# runoff_results.append([50,'NSSA_Runoff_EX-50yr-24hr-AES_BaseRR.res1d'])
# runoff_results.append([100,'NSSA_Runoff_EX-100yr-24hr-AES_BaseRR.res1d'])
# runoff_results.append([200,'NSSA_Runoff_EX-200yr-24hr-AES_BaseRR.res1d'])

In [55]:
deficits = pd.read_csv(map_folder + '\\Deficits.csv',dtype={'Catchment': str})
deficits



,Model,Catchment,Deficit
0,FSA_Base_2060pop_After_First_Nations_VFD_3xADW...,84652,288.011305
1,FSA_Base_2060pop_After_First_Nations_VFD_3xADW...,84653,2956.604835
2,FSA_Base_2060pop_After_First_Nations_VFD_3xADW...,84654,3751.952925
3,FSA_Base_2060pop_After_First_Nations_VFD_3xADW...,84655,1828.174440
4,FSA_Base_2060pop_After_First_Nations_VFD_3xADW...,84656,703.901024
...,...,...,...
959,FSA_Base_2060pop_After_First_Nations_VFD_3xADW...,85159,164.809014
960,FSA_Base_2060pop_After_First_Nations_VFD_3xADW...,85160,36.316172
961,FSA_Base_2060pop_After_First_Nations_VFD_3xADW...,85161,371.828671
962,FSA_Base_2060pop_After_First_Nations_VFD_3xADW...,84922N,737.927969


In [38]:


runoff_stats = []
for i in range(20):
    runoff_stats.append([0,'ForceText',0])
for runoff_result in runoff_results:
    return_period = runoff_result[0]
    result_file = result_folder + '\\' + runoff_result[1]
    print ('Import ' + runoff_result[1])
    res1d = Res1D(result_file)
    for catchment in res1d.data.Catchments:
        muid = catchment.Id
        if not ' - RDI' in muid and not ' - Kinematic wave (B)' in muid:
            peak_runoff = max(list(res1d.query.GetCatchmentValues(muid,'TotalRunOff')))
            runoff_stats.append([return_period,muid,peak_runoff])
            
runoff_stats = pd.DataFrame(runoff_stats,columns=['Return','Catchment','Runoff'])
runoff_stats.to_csv(map_folder + '\\Runoff_Stats.csv',index=False) 






Import FSA_Runoff_EX-2yr-24hr-AES_BaseRR.res1d
Import FSA_Runoff_EX-5yr-24hr-AES_BaseRR.res1d
Import FSA_Runoff_EX-10yr-24hr-AES_BaseRR.res1d
Import FSA_Runoff_EX-25yr-24hr-AES_BaseRR.res1d
Import FSA_Runoff_EX-50yr-24hr-AES_BaseRR.res1d
Import FSA_Runoff_EX-100yr-24hr-AES_BaseRR.res1d
Import FSA_Runoff_EX-200yr-24hr-AES_BaseRR.res1d


In [56]:
return_match_list = []
for i in range(20):
    return_match_list.append(['ForceText','ForceText',0,0,0])

return_periods = [2,5,10,25,50,100,200]
deficits = pd.read_csv(map_folder + '\\Deficits.csv',dtype={'Catchment': str})
runoff_stats
for index, row in deficits.iterrows():
    model = row['Model']    
    catchment = row['Catchment']    
    deficit = row['Deficit']/86400
    return_match = 0
    return_match_runoff = 0
    for i, return_period in enumerate(return_periods):
        runoff = runoff_stats.query("Catchment=='" + catchment + "' & Return==" + str(return_period)).iloc[0,2]
        if deficit >= runoff:
            return_match = return_period
            return_match_runoff = runoff
    return_match_list.append([model,catchment,deficit,return_match_runoff,return_match])
                   
return_match_df = pd.DataFrame(return_match_list,columns=['Model','Catchment','Deficit','Runoff','Return'])
return_match_df.to_csv(map_folder + '\\Return_Period_Match.csv',index=False) 
return_match_df

,Model,Catchment,Deficit,Runoff,Return
0,ForceText,ForceText,0.000000,0.000000,0
1,ForceText,ForceText,0.000000,0.000000,0
2,ForceText,ForceText,0.000000,0.000000,0
3,ForceText,ForceText,0.000000,0.000000,0
4,ForceText,ForceText,0.000000,0.000000,0
...,...,...,...,...,...
979,FSA_Base_2060pop_After_First_Nations_VFD_3xADW...,85159,0.001908,0.000000,0
980,FSA_Base_2060pop_After_First_Nations_VFD_3xADW...,85160,0.000420,0.000000,0
981,FSA_Base_2060pop_After_First_Nations_VFD_3xADW...,85161,0.004304,0.002407,200
982,FSA_Base_2060pop_After_First_Nations_VFD_3xADW...,84922N,0.008541,0.000000,0


In [40]:
deficits

,Model,Catchment,Deficit
0,FSA_Base_2020pop_2p5xADWF.mdb,84652,214.369795
1,FSA_Base_2020pop_2p5xADWF.mdb,84653,1636.960227
2,FSA_Base_2020pop_2p5xADWF.mdb,84654,2153.737887
3,FSA_Base_2020pop_2p5xADWF.mdb,84655,1156.079876
4,FSA_Base_2020pop_2p5xADWF.mdb,84656,499.620119
...,...,...,...
6739,FSA_Base_2020pop_5xADWF_VFD.mdb,85159,355.164554
6740,FSA_Base_2020pop_5xADWF_VFD.mdb,85160,88.122408
6741,FSA_Base_2020pop_5xADWF_VFD.mdb,85161,840.238114
6742,FSA_Base_2020pop_5xADWF_VFD.mdb,84922N,1404.231344


In [41]:
for spec in specs:
    times_adwf = str(spec[0])
    deficit = spec[1]
    result_file =  spec[2]
    wwtp = spec[3]
    suffix = spec[4]
    
    print(suffix)

VFD


In [57]:


pipes = []
for i in range(20):
    pipes.append(['ForceText','ForceText',0,0,0])

res1d_DWF = Res1D(dwf_file_path)

for spec in specs:
    times_adwf = str(spec[0])
    deficit = spec[1]
    result_file =  spec[2]
    wwtp = spec[3]
    suffix = spec[4]
    df_theory = pd.read_excel(df_theory_sheet)
    df_theory = df_theory[['DateTime','Discharge']]
    df_theory.set_index('DateTime', inplace=True)
    df_theory['DWF_Hourly'] = (df_theory['Discharge']).rolling('1h').mean()
    df_theory[times_adwf + 'xADWF'] = (df_theory['Discharge']+deficit)
    df_theory[times_adwf + 'xADWF_Hourly'] = df_theory[times_adwf + 'xADWF'].rolling('1h').mean()
    
    res1d_XADWF = Res1D(result_folder + '\\' + result_file)
    
    #Create csv file for map input.
    for i, reach in enumerate(res1d_DWF.data.Reaches):
        pipe = reach.Id
        pipe = pipe[:pipe.rfind('-')]

        adwf = sum(list(res1d_DWF.query.GetReachEndValues(pipe, "Discharge"))[-289:])/289
        pf = max(list(res1d_XADWF.query.GetReachEndValues(pipe, "Discharge")))
        try: 
            pf_per_adwf = pf / adwf
        except:
            pf_per_adwf = np.nan
        pipes.append([result_file,pipe,adwf,pf,pf_per_adwf])
    
    #Create plots
    df = pd.DataFrame(index=res1d_DWF.time_index)[288:]
    df['DWF'] = list(res1d_DWF.query.GetReachEndValues(wwtp_pipe, "Discharge"))[288:]
    df['DWF (Inflow)'] = df_theory['DWF_Hourly']
    df[times_adwf + ' * ADWF (result)'] = list(res1d_XADWF.query.GetReachEndValues(wwtp_pipe, "Discharge"))[288:]
    df[times_adwf + ' * ADWF Deficit'] = deficit
#     df[times_adwf + ' * ADWF (DWF Result + Deficit)'] = df['DWF'] + df[times_adwf + ' * ADWF Deficit']
    df[times_adwf + ' * ADWF (Inflow)'] = df_theory[times_adwf + 'xADWF_Hourly']

    fig = go.Figure()

    for column in df.columns:
        fig.add_trace(go.Scatter(x=df.index, 
                                     y = df[column], 
                                     mode='lines',name=column)) 
        
    title_suffix = ''
    if len(suffix) > 0: 
        title_suffix = ', ' + suffix
    fig.update_layout(title ={'text' : 'Inflow to ' + wwtp + ' at ' + times_adwf + ' times ADWF' + title_suffix})
    fig.show()

    wwtp_filename = wwtp.replace("'","").replace(" ","_")
    
    file_suffix = ''
    if len(suffix) > 0: 
        file_suffix = '_' + suffix
        
    with open(map_folder + '\\Inflow_To_' + wwtp_filename + '_' + times_adwf + 'xADWF' + file_suffix + '.html', 'w') as f:
        f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.close()
        
pipe_df = pd.DataFrame(pipes,columns=['Result','Pipe','ADWF','INI','INI_Per_ADWF'])
pipe_df.to_csv(map_folder + "\\INI_ADWF.csv",index=False)
pipe_df


,Result,Pipe,ADWF,INI,INI_Per_ADWF
0,ForceText,ForceText,0.000000,0.000000,0.000000
1,ForceText,ForceText,0.000000,0.000000,0.000000
2,ForceText,ForceText,0.000000,0.000000,0.000000
3,ForceText,ForceText,0.000000,0.000000,0.000000
4,ForceText,ForceText,0.000000,0.000000,0.000000
...,...,...,...,...,...
7435,FSA_DWF_2060pop_SealPoco_FN_VFD_3ADWF_Base.res1d,Orifice:SCSO_SouthGate,0.179389,0.544835,3.037172
7436,FSA_DWF_2060pop_SealPoco_FN_VFD_3ADWF_Base.res1d,Orifice:TimerOrifice_AnnacisQ,0.001952,0.141323,72.406836
7437,FSA_DWF_2060pop_SealPoco_FN_VFD_3ADWF_Base.res1d,Orifice:TimerOrifice_NW2Q,0.001952,0.131988,67.623905
7438,FSA_DWF_2060pop_SealPoco_FN_VFD_3ADWF_Base.res1d,Orifice:V-001A,0.000000,2.277304,NaN


In [43]:
suffix

'VFD'

In [44]:
res1d_DWF.query.GetReachEndValues(wwtp_pipe, "Discharge")

<System.Single[] object at 0x00000257D7D18160>

In [45]:
# pipes = []
# for i, reach in enumerate(r[1].data.Reaches):
#     muid = reach.Id
#     muid = muid[:muid.rfind('-')]
#     pipes.append(muid)

# for pipe in pipes[:1]:    
#     fig = go.Figure()
#     for r in result_list:
#         fig.add_trace(go.Scatter(x=r[1].time_index, 
#                                      y = list(r[1].query.GetReachStartValues(pipe, "WaterLevel")), 
#                                      mode='lines',name=r[0]))

#     with open(result_folder + "\\HTMLs\\Discharge_Pipe_" + pipe + ".html", 'w') as f:
#         f.write("<h1>Discharge in pipe '" + pipe + "'</h1>")
#         f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
#     f.close()

In [46]:
# df.to_csv(r'J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\Threes_Times_ADWF\Model\SA_Maps\Inflow_To_Annacis.csv')

In [47]:
list(res1d_DWF.query.GetReachEndValues(pipe, "Discharge"))[288:]

[0.05029362,
 0.05034474,
 0.04965484,
 0.04965099,
 0.04899051,
 0.04866492,
 0.04819209,
 0.04783242,
 0.04739484,
 0.04702748,
 0.04666647,
 0.04634026,
 0.04601781,
 0.04569228,
 0.04538396,
 0.04504895,
 0.04477118,
 0.04443192,
 0.04451752,
 0.04429581,
 0.04428172,
 0.04402303,
 0.04387503,
 0.04372858,
 0.04319127,
 0.04239391,
 0.04233594,
 0.04248775,
 0.04245981,
 0.04218965,
 0.04202402,
 0.04167464,
 0.0411348,
 0.04130631,
 0.03944813,
 0.03892892,
 0.03975513,
 0.039939,
 0.03754194,
 0.03997564,
 0.04114344,
 -0.366019,
 -0.1643804,
 -0.0689206,
 -0.03580871,
 0.009046983,
 0.04491975,
 0.04531765,
 0.04151064,
 0.04034803,
 0.04239706,
 0.04537532,
 0.04673574,
 0.04710525,
 0.04715023,
 0.04765908,
 0.04829295,
 0.04916446,
 0.05037061,
 0.0510319,
 0.05217116,
 0.0532846,
 0.05457109,
 0.05490292,
 0.05678549,
 0.05793886,
 0.0598238,
 0.06066978,
 0.06205276,
 0.06329507,
 0.06459287,
 0.06644477,
 0.06721918,
 0.06849933,
 0.06949931,
 0.07103966,
 0.07192022,
 0.0

In [48]:
res1d_DWF

In [49]:
dwf_file_path

'J:\\SEWER_AREA_MODELS\\FSA\\03_SIMULATION_WORK\\Port_Coquitlam_PS_VFD\\Model_3ADWF\\FSA_DWF_2060pop_SealPoco_FN_VFD_Base.res1d'

In [50]:
pf

0.3980225